### 기본 Prompt 구조 이해
✅ prompt에는 3가지 종류의 역할(role)이 존재
1. User Prompt : 사용자가 LLM 모델에게 전달하는 prompt
2. Assistant Prompt : LLM 모델이 응답하는 prompt
3. System Prompt : 사용자 Prompt 이전에 정의되는 전제 및 규칙 prompt

- System Prompt 예시
    - 출력 형태 지정 (ex. json, ppt 등)
    - 페르소나(투자전문가, 예술가 등) 및 어조(공손한, 전문적인 등) 설정
    - 모델이 지켜야 할 규칙 설정
    - 기타 Base가 되는 외부 정보 주입
ChatGPT 포함 웬만한 LLM 모델들은 prompt 입력 시, 기본적으로 개발자가 지정해 둔 System Prompt가 붙어서 동작

In [1]:
from openai import OpenAI
from getpass import getpass

In [2]:
MY_API_KEY = getpass("OpenAI API Key >> ")

In [3]:
client = OpenAI(api_key = MY_API_KEY)

In [4]:
content = "당신은 물리학 선생님입니다. 초등학생에게 설명하듯이 아주 쉽고 친근하게 설명해야 합니다."
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role" : "system", "content" : content},
        {"role" : "user", "content" : "왜 하늘은 파란색인가요?"}
    ],
    temperature = 0
)
print(completion.choices[0].message.content)

하늘이 파란색인 이유는 바로 '산란' 때문이에요! 태양에서 나오는 빛은 다양한 색깔을 가지고 있는데, 그 중에서도 파란색 빛이 하늘에 가장 많이 흩어지게 되요. 이 파란색 빛이 하늘 곳곳에 흩어지면서 우리 눈에 파란색으로 보이게 되는 거에요. 그래서 하늘은 파란색으로 보이는 거죠!


In [5]:
content = "당신은 물리학 선생님입니다. 초등학생에게 설명하듯이 아주 쉽고 친근하게 설명해야 합니다. 왜 하늘은 파란색인가요?"
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : content}],
    temperature = 0
)
print(completion.choices[0].message.content)

하늘은 파란색인 이유는 태양빛이 하늘을 통과할 때 빛의 색깔 중 파란색이 가장 많이 흩어지기 때문이에요. 태양빛은 다양한 색깔을 가지고 있는데, 이 중 파란색 빛은 다른 색깔보다 더 짧은 파장을 가지고 있어요. 그래서 파란색 빛은 공기 분자와 충돌할 때 더 많이 흩어지게 되고, 우리 눈에 파란색으로 보이게 되는 거에요. 그래서 하늘은 파란색으로 보이는 거죠!


#### Stream 객체 살펴보기
- stream = True로 설정 시, 모델이 문장을 모두 완성하여 출력하는 것이 아니라, 각 청크(토큰들의 집합) 별 완성되는 대로 바로 출력

In [7]:
stream = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : "왜 하늘은 파란색인가요?"}],
    temperature = 0,
    stream = True
)
# stream = True 설정 시, 응답 객체가 ChatCompletion이 아닌, ChatCompletionChunk로 반환되고, 각 청크 마다 응답을 실시간으로 받게 됨
print(type(stream))
print("=" * 30)

for i in stream :
    # print(i)
    content = i.choices[0].delta.content
    if content is not None :
        print(content, end = "")

<class 'openai.Stream'>
하늘은 파란색으로 보이는 이유는 대기 중의 분자들이 햇빛을 흡수하고 다시 방출할 때 파란색 빛이 가장 많이 방출되기 때문입니다. 태양으로부터 오는 햇빛은 다양한 색상의 빛으로 구성되어 있지만, 대기 중의 분자들이 파란색 빛을 더 많이 흡수하고 방출하기 때문에 하늘은 파란색으로 보이는 것입니다.

#### 프롬프트 엔지니어링에서 기본적으로 생각해야 될 점
- 생성형 AI 모델의 출력은 입력 값에 대한 의존도가 매우 높음
    - 잘 입력한 것 같으나, 원하는 결과가 나오지 않는다면 입력이 모호하거나 응답에 필요한 내용이 빠졌을 수도 있음
    - 그게 아닌 경우 모델의 능력으로 해결하기 힘든 요청일 수도 있음
- 자연어 질의를 기반으로 하기 때문에 절대적으로 성능이 좋은 prompt가 존재하지 않음
    - 성능이 좋다는 것은 사용자가 만족해야 한다는 것인데, 이는 매우 주관적이고 판단하기 어려움
    - 정확한 응답이 좋을 수도 있고, 창의적인 응답이 좋을수도 있음 (temperature로 조정 가능)
- 즉, 프롬프트 엔지니어링은 Task에 맞는 여러 번의 테스트가 중요하고, 이를 통한 반복적인 개선이 필요

### LLM 모델의 Task에 따른 평가 지표들을 알아보고, 대화 및 Q&A Task에 실제로 적용해보자.
#### ✅ 전통적인 Language Model 평가 지표
1. MMLU (Massive Multitask Language Understanding)
    - 다양한 분야에 대한 질문 후, 정답을 찾아내는 객관식 시험
2. HellaSwag
    - 문장들을 주고 이어지는 마지막 문장으로 가장 적합한 4개의 문장 중 하나를 고르는 문제
#### ✅ 대화 및 Q&A task에 적합한 평가 기법 방식
1. Human Based Method : 사람이 평가하는 방법
2. Model Based Evaluation : LLM 모델이 평가하는 방법
3. Code Based Evaluation : 코드로 평가하는 방법
#### 1) Human Based Method
- 전문가 블라인드 A/B 테스트
- 2가지 모델의 답변 중에서 더 좋은 답변을 선택하는 방법
- 명확한 결과로 성능을 판단하기 쉬움
- 많은 인력과 그에 따른 비용, 시간이 필요함
##### LMSys사 Chatbot Arena
- 대표적인 Human Based Method 중 하나로, 동일한 질문에 대해 2개 모델의 답변을 보고 승/패/무 투표 이후에 모델 명을 공개하는 방식
- 투표한 결과에 대한 모델들의 랭킹도 확인 가능 (현재까지 140만 번 이상의 테스트 진행)
#### 2) Model Based Evaluation
고성능(일반적으로 GPT4 이상) strong LLM을 통해 평가하는 방법 (LLM-as-a-judge)
- 실제로 사람이 평가하는 것과 굉장히 유사하고 일치율이 높다는 논문 결과들이 나오고 있음
##### 평가 방식 3가지
1. Pairwise Comparison
- 2개의 모델에 같은 질문을 하고, 2개의 답변을 받아 어떤 답변이 좋은지 혹은 무승부인지 출력
2. Single Answer Grading
- 질문과 답변이 있을 때 점수를 매기는 것
3. Reference-Guided Grading
- 예시 답변을 주고, 얼마나 유사한 지에 대한 점수를 매기는 것
#### 3) Code Based Evaluation
- Accuracy, Precision, Recall, F1score 등 일반적인 평가 지표들을 사용
- ROUGE : 요약 모델의 성능을 평가하는 지표 (모델이 요약한 내용과 실제 정답 요약본의 일치율을 비교)
- BLEU : 번역 모델 평가 지표
- Human Based 및 Model Based에 비해 실제 사용자들의 만족과 크게 일치하지는 않음

- 참조 논문 : Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena, 2023
- 논문 내용으로 LLM 모델의 평가가 사람의 평가와 일치율이 점점 높아지고 있다는 것을 증명 (GPT4의 경우 사람과 85% 수준의 일치율)
- 즉, 이는 기계의 판단이 인간의 판단에 도움이 되며, 기계를 통한 자동화된 평가 프로세스 개발이 가능하다는 것을 시사함 (실제로 해당 논문에서 인간은 75%의 사례에서 기계의 판단이 합리적이라 생각했고, 34%의 사례에서 자신의 선택을 바꿀 의향이 있다고 대답 함)

#### 위 논문을 참고하여, 대화 및 Q&A에 적용해보자.
- gpt-3.5와 gpt-4의 응답을 비교하여 gpt-4o에게 어떤 응답이 더 나은지를 판단하게 함
- 평가를 위한 프롬프트는 MT-Bench and Chatbot Arena 논문의 Figure 5 부분 참조  

In [8]:
question = "하늘은 왜 파란색인가요?"

In [9]:
# gpt-3.5-turbo
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : question}],
    temperature = 0
)
answer_a = completion.choices[0].message.content
print(answer_a)

하늘이 파란색인 이유는 태양빛이 대기 중의 공기 분자들과 상호 작용하여 발생하는 현상 때문입니다. 태양빛은 다양한 색상의 빛으로 구성되어 있지만, 대기 중의 공기 분자들은 파란색 빛을 더 많이 흡수하고 다른 색상의 빛을 더 많이 반사합니다. 이러한 현상으로 인해 우리가 보는 하늘은 파란색으로 보이게 됩니다.


In [10]:
# gpt-4-turbo
completion1 = client.chat.completions.create(
    model = "gpt-4-turbo",
    messages = [{"role" : "user", "content" : question}],
    temperature = 0
)
answer_b = completion1.choices[0].message.content
print(answer_b)

하늘은 왜 파란색인가요?라는 질문에 대한 답은 대기 중의 빛 산란 현상 때문입니다. 태양으로부터 오는 빛은 사실 모든 색깔을 포함하고 있는데, 이 빛이 지구 대기에 들어오면 대기 분자와 작은 입자들에 의해 산란됩니다. 이 과정에서 파란색 빛은 다른 색깔의 빛보다 더 많이 산란됩니다.

이유는 파란색 빛의 파장이 상대적으로 짧기 때문입니다. 파장이 짧은 빛은 다른 색의 빛보다 산란되기 쉽습니다. 이러한 현상을 '레일리 산란'이라고 하며, 이로 인해 대기를 통과하는 동안 파란색 빛이 다른 색보다 더 효과적으로 산란되어 우리 눈에는 하늘이 파란색으로 보이게 됩니다.

태양이 지평선에 가까울 때, 즉 일출이나 일몰 때에는 하늘이 빨강, 주황 또는 분홍색으로 보이는 경우가 있는데, 이는 태양 빛이 대기를 통과하는 거리가 더 길어지면서 파란색 빛이 더 많이 산란되고, 빨간색과 주황색 빛이 상대적으로 덜 산란되어 그 색이 더 강하게 나타나기 때문입니다.


In [11]:
# 두 모델의 응답 평가를 위해 논문의 프롬프트 예시를 활용
prompt = f"""
[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Question]
{question}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]
"""

print(prompt)


[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Question]
하늘은 왜 파란색인가요?
[The Start of Assista

In [12]:
# 가장 최신 버전 gpt-4o로 각 모델들의 응답을 평가
completion2 = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role" : "user", "content" : prompt}],
    temperature = 0
)
print(completion2.choices[0].message.content)

Assistant A provides a concise explanation of why the sky is blue, mentioning the interaction of sunlight with air molecules and the selective absorption and reflection of blue light. However, the explanation is somewhat simplified and contains a minor inaccuracy: it states that air molecules absorb blue light more, which is not correct. The correct phenomenon is scattering, not absorption.

Assistant B offers a more detailed and accurate explanation, correctly identifying Rayleigh scattering as the reason for the blue sky. It explains that blue light is scattered more due to its shorter wavelength. Additionally, Assistant B provides extra context by explaining why the sky appears red or orange during sunrise and sunset, which adds depth to the response.

Considering the factors of accuracy, depth, and detail, Assistant B provides a more comprehensive and correct answer.

Final verdict: [[B]]


### 장단점 비교
1. Human Based Method
- 통제된 환경을 가정했을 때 사람이 직접 평가한 방법이라서 안정적이고 신뢰 가능함
- 불특정 다수의 경우 약간의 응답 편차가 발생할 수 있음
- 전문 분야의 경우 해당 전문가가 아닌 일반인이 평가할 경우 정확도 및 평가 속도가 낮아질 수 있음

2. Model Based Evaluation
- 사람의 평가와 어느 정도 유사한 수준의 판단을 내릴 수 있음
- 평가를 위해 API 호출이 필요한데, 평가 데이터의 양이 굉장히 많을 경우 수 백만원 이상은 금방 넘어갈 수 있음

3. Code Based Evaluation
- 위 방법들에 비해 인력 비용, 모델 호출 비용 등이 없는 무료 평가 방법
- Task에 따라 더 정확할 수도 있고, 그러지 않을 수도 있음
- 사람에게 적합한 답변을 선택하는 데 있어서는 신뢰도가 상대적으로 떨어지는 편

**실제 서비스의 관점에서 봤을 때 결국은 사용자의 피드백이 가장 중요하므로 상황에 따라 여러가지 기법들을 활용해 보는 것이 좋음**

### 프롬프트 엔지니어링 고도화 기법
1. Few-shot
   - 참고할 수 있는 질의/응답 예시나 사례들을 프롬프트에 추가하여 질의
2. Chain-of-thought
   - 모델에게 문제 해결 과정을 단계 별로 알려주면서 질의 (보통 Few-shot과 함께 사용)

### 1. Few-shot

In [13]:
# zero-shot
prompt = """Q : Who wrote the book 'HARRY POTTER'?"""
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : prompt}],
    temperature = 0
)
print(completion.choices[0].message.content)

J.K. Rowling wrote the book 'Harry Potter'.


In [14]:
# one-shot
prompt = """
Answer these questions :
Q : Who wrote the book 'HARRY POTTER'?

Below is an example for your reference.
Q : Who sang 'One Call Away'?
A : Charlie Puth
"""
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : prompt}],
    temperature = 0
)
print(completion.choices[0].message.content)

Q : Who wrote the book 'HARRY POTTER'?
A : J.K. Rowling


- 우리가 제시해 준 예시를 잘 참고하여 질문에 대해 대답하는 것을 확인할 수 있음

### 2. Chain-of-thought
- 마이크로소프트에서 활용한 예시 프롬프트를 사용

In [15]:
# Base 프롬프트로 진행
prompt = """Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?"""
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : prompt}],
    temperature = 0
)
print(completion.choices[0].message.content)
# 정답은 1개여야 하는데 5개로 잘못 계산함

Alice would have 5 apples - 3 thrown + 2 given to Bob + 1 received from Bob = 5 apples. 

So, Alice still has 5 apples.


#### CoT + one-shot

In [16]:
prompt = """
Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?
Below is an example for your reference.
Lisa has 7 apples, throw 1 apple. give 4 apples to Bart and Bart gives one back :
7 - 1 = 6
6 - 4 = 2
2 + 1 = 3
"""
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : prompt}],
    temperature = 0
)
print(completion.choices[0].message.content) # 드디어 정답

Alice has 5 apples - 3 thrown = 2 apples
2 apples - 2 given to Bob = 0 apples
Bob gives 1 apple back, so Alice now has 0 + 1 = 1 apple.


In [17]:
prompt = """Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?"""
completion = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role" : "user", "content" : prompt}],
    temperature = 0
)
print(completion.choices[0].message.content)

Let's break down the sequence of events to determine how many apples Alice has:

1. Alice starts with 5 apples.
2. Alice throws 3 apples. (Assuming these apples are no longer in her possession)
   - Apples left: 5 - 3 = 2 apples
3. Alice gives 2 apples to Bob.
   - Apples left: 2 - 2 = 0 apples
4. Bob gives one apple back to Alice.
   - Apples left: 0 + 1 = 1 apple

So, after all these transactions, Alice has 1 apple.


#### 또 다른 프롬프트 고도화 예시
- KMMLU 논문의 예시 프롬프트 템플릿 사용

In [18]:
# 실제 정답은 B = 6
question = "x, y가 세 부등식 y ≤ x+3, y ≤ -4x+3, y ≥ 0을 만족할 때, x+y의 최댓 값을 M, 최솟 값을 m이라 하면 M-m의 값은?"
A = 4
B = 6
C = 8
D = 10

In [19]:
prompt = f"""
{question}
A. {A}
B. {B}
C. {C}
D. {D}
정답：
"""
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : prompt}],
    temperature = 0
)
print(completion.choices[0].message.content)

C. 8


In [20]:
prompt = f"""
{question}
A. {A}
B. {B}
C. {C}
D. {D}
정답：
"""
completion = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role" : "user", "content" : prompt}],
    temperature = 0
)
print(completion.choices[0].message.content)

주어진 부등식들을 분석하여 \(x\)와 \(y\)의 범위를 찾아보겠습니다.

1. \(y \leq x + 3\)
2. \(y \leq -4x + 3\)
3. \(y \geq 0\)

이 세 부등식을 만족하는 영역을 구하기 위해 각 부등식의 경계선을 그려보겠습니다.

1. \(y = x + 3\)
2. \(y = -4x + 3\)
3. \(y = 0\)

이 세 직선이 만나는 점들을 찾아보겠습니다.

1. \(y = x + 3\)와 \(y = -4x + 3\)의 교점을 구합니다.
   \[
   x + 3 = -4x + 3 \implies 5x = 0 \implies x = 0
   \]
   \[
   y = 0 + 3 = 3
   \]
   따라서 교점은 \((0, 3)\)입니다.

2. \(y = x + 3\)와 \(y = 0\)의 교점을 구합니다.
   \[
   0 = x + 3 \implies x = -3
   \]
   따라서 교점은 \((-3, 0)\)입니다.

3. \(y = -4x + 3\)와 \(y = 0\)의 교점을 구합니다.
   \[
   0 = -4x + 3 \implies 4x = 3 \implies x = \frac{3}{4}
   \]
   따라서 교점은 \(\left(\frac{3}{4}, 0\right)\)입니다.

이제 이 세 점 \((0, 3)\), \((-3, 0)\), \(\left(\frac{3}{4}, 0\right)\)을 이용하여 \(x + y\)의 값을 계산해보겠습니다.

1. \((0, 3)\)에서 \(x + y = 0 + 3 = 3\)
2. \((-3, 0)\)에서 \(x + y = -3 + 0 = -3\)
3. \(\left(\frac{3}{4}, 0\right)\)에서 \(x + y = \frac{3}{4} + 0 = \frac{3}{4}\)

따라서 \(x + y\)의 최댓값은 3이고, 최솟값은 -3입니다. \(M - m\)을 계산하면:
\[
M - m = 3 - (-3) = 6
\]

정답은 B. 6입

#### GPT-3.5 모델 고도화 작업
- 페르소나 적용 + 영문으로 프롬프트 작성

In [21]:
prompt = f"""
You are a Professional in Mathmatics. Below id given a math question in Korean.
{question}
A. {A}
B. {B}
C. {C}
D. {D}
Answer：
"""
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : prompt}],
    temperature = 0
)
print(completion.choices[0].message.content)

C. 8


#### Chain of Thought 적용

In [22]:
prompt = f"""
You are a Professional in Mathmatics. Below id given a math question in Korean.
You have to think carefully and step by step about the question and choose one of four given answer.
Only one of them is true.

{question}
A. {A}
B. {B}
C. {C}
D. {D}
Answer：
"""
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [{"role" : "user", "content" : prompt}],
    temperature = 0
)
print(completion.choices[0].message.content)

B. 6

To find the maximum and minimum values of x+y, we need to find the intersection points of the three inequalities.

First, let's find the intersection points of y ≤ x+3 and y ≤ -4x+3:
x+3 = -4x+3
5x = 0
x = 0

When x = 0, y = 3

So, the intersection point of these two inequalities is (0, 3).

Next, let's find the intersection points of y ≤ x+3 and y ≥ 0:
x+3 = 0
x = -3

When x = -3, y = 0

So, the intersection point of these two inequalities is (-3, 0).

Now, let's find the intersection points of y ≤ -4x+3 and y ≥ 0:
-4x+3 = 0
-4x = -3
x = 3/4

When x = 3/4, y = 0

So, the intersection point of these two inequalities is (3/4, 0).

Now, we have three intersection points: (0, 3), (-3, 0), and (3/4, 0).

To find the maximum value of x+y, we need to find the point that is farthest from the origin, which is (0, 3). So, the maximum value of x+y is 3.

To find the minimum value of x+y, we need to find the point that is closest to the origin, which is (-3, 0). So, the minimum value of x+y

- gpt-3.5-turbo의 경우, CoT까지 적용을 하니 정답을 맞추는 것을 볼 수 있음
- 4o 모델의 경우 zero-shot으로도 좋은 성능을 내는 것을 볼 수 있지만, 비용 측면에서는 3.5 모델이 10배 정도 저렴하기 때문에 이런 부분을 고려하여 3.5 모델에서 프롬프트를 고도화 시키는 작업이 선행 되어야 함

### 💡 프롬프트 엔지니어링 정리
- 추가 학습이 없어도 성능 개선의 가능성이 있기 때문에 가성비가 굉장히 좋음
- 더 좋은 모델을 사용하면 프롬프트 엔지니어링 없이도 해결되는 케이스들이 다수 존재
- 그러나 모델의 알고리즘이 완전히 바뀌게 되면 기존에 사용했던 프롬프팅 방식들이 달라질 수 있음 (변동성 up)
- 프롬프트 엔지니어링은 모델의 응답 성능을 높일 수 있는 수많은 방법론 중 하나이며, 처음에는 프롬프트 엔지니어링으로 성능 향상을 시도해 보고 부족하면 RAG, Fine-tuning과 같은 순서로 다른 방법을 시도해 볼 수 있음